In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

#from sklearn.metrics import accuracy_score, precision_score, recall_score
#from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, optimizers, callbacks, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [3]:
model_sound = models.load_model('model_sound/')
model_underhang = models.load_model('model_underhang/')

In [ ]:
def resnet_pseudo():
    branch_output=[]  
     
    inputA = tf.keras.Input(shape=(config['img_size'], config['img_size'], 3))
    augmented = data_augmentation(inputA)
    #x = layers.experimental.preprocessing.Rescaling(1./255)(x)
        
   #inception_v3   
   #mobilenet_v2
    #mobilenet
    #densenet
    
    x = tf.keras.layers.experimental.preprocessing.RandomCrop(160,160,3)(augmented) 
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x) 
    
      #DenseNet121
        #ResNet50
        #Xception
        #MobileNet
        #MobileNetV2
        #InceptionV3
    
    model = keras.models.load_model('../input/mobilenetv2-imagenet-weights-input160x160x3/MobileNetV2_imagenet.hdf5')
    
    model.trainable = False
    
    x = model(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.25)(x)
    #x = layers.Dense(512, activation='relu')(x)
    #x = layers.Dropout(0.5)(x)
    branch_output.append(x) 
    
    #transformer
    trans = layers.experimental.preprocessing.Normalization()(augmented)
    trans = create_vit_classifier()(trans)
    trans = layers.Dropout(0.1)(trans)
    branch_output.append(trans) 
    
    inputB = keras.Input(shape=(df_train.shape[-1],)) 
    x = layers.Dense(128, activation='relu')(inputB)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    branch_output.append(x) 
    
    combined = layers.Concatenate()(branch_output)
    
    out = layers.Dense(1024, activation='relu')(combined)
    out = layers.Dropout(0.25)(out)
    out = layers.Dense(256, activation='relu')(out)
    out = layers.Dense(1)(out)
    
    model_final = tf.keras.Model(inputs = [inputA, inputB],outputs=out)

    model_final.compile(loss='mean_squared_error',
                #loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=config['learning_rate']),
                    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")],
                       )
    
    return model_final